Import libraries

In [1]:
import pandas as pd

In [2]:
import data_prep as dp

Import Data

In [3]:
dat_val, dat_tr = dp.load_train_split()

Path to dataset files: /Users/anabellafalk/.cache/kagglehub/datasets/kartik2112/fraud-detection/versions/1


### Prepare Data

In [4]:
# Initialize relevant columns
model_cols = ['category', 'amt', 'trans_month', 'trans_year']
identifier_cols = ['state', 'age_at_trans', 'city_pop', 'job_bin']
other_cols = ['distance']

#### Extract Date Info

In [5]:
# Create month column
dp.extract_month(dat_tr)
dp.extract_month(dat_val)

In [6]:
# Create year column
dp.extract_year(dat_tr)
dp.extract_year(dat_val)

In [7]:
# Create month and year column
dp.extract_month_year(dat_tr)
dp.extract_month_year(dat_val)

#### Extract Age

In [8]:
# Calculate and create age column
dp.extract_age(dat_tr)
dp.extract_age(dat_val)

#### Bin Jobs

In [9]:
# Fit binning on training data
job_bin_map = dp.fit_job_bins(dat_tr)

In [11]:
# Apply bins to training data
dp.bin_jobs(dat_tr, job_bin_map)
dp.bin_jobs(dat_val, dp.fit_job_bins(dat_val))

#### Extract Distance

In [12]:
# Calculate and create distance column
dp.extract_distance(dat_tr)
dp.extract_distance(dat_val)

#### Scale Numeric Data

#### Encode Categorical Data

##### Encode Category

In [13]:
category_encoder = dp.fit_col_encoder(dat_tr, 'category')
category_enc_tr = dp.encode_col(dat_tr, 'category', category_encoder)
category_enc_val = dp.encode_col(dat_val, 'category', category_encoder)
category_enc_tr

,category.entertainment,category.food_dining,category.gas_transport,category.grocery_net,category.grocery_pos,category.health_fitness,category.home,category.kids_pets,category.misc_net,category.misc_pos,category.personal_care,category.shopping_net,category.shopping_pos,category.travel
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319617,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319618,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319619,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


##### Encode Year

In [14]:
year_encoder = dp.fit_col_encoder(dat_tr, 'trans_year')
year_enc_tr = dp.encode_col(dat_tr, 'trans_year', year_encoder)
year_enc_val = dp.encode_col(dat_val, 'trans_year', year_encoder)
year_enc_tr

,trans_year.2019,trans_year.2020
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
319617,0.0,1.0
319618,0.0,1.0
319619,0.0,1.0
319620,0.0,1.0


##### Encode State

In [17]:
state_encoder = dp.fit_col_encoder(dat_tr, 'state')
state_enc_tr = dp.encode_col(dat_tr, 'state', state_encoder)
state_enc_val = dp.encode_col(dat_val, 'state', dp.fit_col_encoder(dat_val, 'state'))
state_enc_tr

,state.AK,state.AL,state.AR,state.AZ,state.CA,state.CO,state.CT,state.DC,state.FL,state.GA,...,state.SC,state.SD,state.TN,state.TX,state.VA,state.VT,state.WA,state.WI,state.WV,state.WY
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Fit Models

#### Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
rfc = RandomForestClassifier()

In [32]:
rfc_model_dat = pd.concat([
    dat_tr[['is_fraud'] + ['amt'] + identifier_cols[1:3] + other_cols].reset_index(drop=True),
    category_enc_tr.reset_index(drop=True),
    year_enc_tr.reset_index(drop=True),
    #state_enc_tr.reset_index(drop=True)
], axis=1)

In [33]:
rfc.fit(rfc_model_dat.drop('is_fraud', axis=1), rfc_model_dat['is_fraud'])

RandomForestClassifier()

In [34]:
rfc.score(rfc_model_dat.drop('is_fraud', axis=1), rfc_model_dat['is_fraud'])

1.0

In [31]:
identifier_cols

['state', 'age_at_trans', 'city_pop', 'job_bin']

In [35]:
rfc_val_dat = pd.concat([
    dat_val[['is_fraud'] + ['amt'] + identifier_cols[1:3] + other_cols].reset_index(drop=True),
    category_enc_val.reset_index(drop=True),
    year_enc_val.reset_index(drop=True),
    #state_enc_val.reset_index(drop=True)
], axis=1)

In [36]:
rfc.score(rfc_val_dat.drop('is_fraud', axis=1), rfc_val_dat['is_fraud'])

0.9970932999540455

#### SVM

scale features, < 100, use kernel, balance classes

#### XGBoost

#### Logistic Regression